In [1]:
!pip -q install gdown

In [2]:
MODEL_PATH_URL = 'https://drive.google.com/file/d/1sYNP1qaiEpmchRlUHxKjeRbxe05SK1FC/view?usp=drive_link'
SCALER_PATH_URL = 'https://drive.google.com/file/d/1OeBqibnrAe-2f6LV59Hki3xHjm1F53YL/view?usp=drive_link'
FEATURES_PATH_URL = 'https://drive.google.com/file/d/1bT0xYstATzLYOwSIol9V_zoFJaGlpSBQ/view?usp=drive_link'

In [3]:
!gdown --fuzzy -O random_forest_final_model.pth "{MODEL_PATH_URL}"
!gdown --fuzzy -O scaler.pkl "{SCALER_PATH_URL}"
!gdown --fuzzy -O feature_names.pkl "{FEATURES_PATH_URL}"

Downloading...
From (original): https://drive.google.com/uc?id=1sYNP1qaiEpmchRlUHxKjeRbxe05SK1FC
From (redirected): https://drive.google.com/uc?id=1sYNP1qaiEpmchRlUHxKjeRbxe05SK1FC&confirm=t&uuid=e5720f36-e761-4b32-9085-cbfc1b7c7c3b
To: /content/random_forest_final_model.pth
100% 395M/395M [00:06<00:00, 60.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OeBqibnrAe-2f6LV59Hki3xHjm1F53YL
To: /content/scaler.pkl
100% 5.95k/5.95k [00:00<00:00, 22.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bT0xYstATzLYOwSIol9V_zoFJaGlpSBQ
To: /content/feature_names.pkl
100% 2.17k/2.17k [00:00<00:00, 11.5MB/s]


In [ ]:
# 載入所需的函式庫
import gradio as gr
import pandas as pd
import numpy as np
import joblib
import os
import requests
import datetime
import time

In [ ]:
# 假設您在 Colab 環境中執行，並使用 userdata 獲獲取 API Key
try:
    from google.colab import userdata
except ImportError:
    # 如果不在 Colab 環境，定義一個虛擬的 userdata 函數
    class MockUserData:
        def get(self, key):
            # 在非 Colab 環境中，請在此處手動填寫您的 API Key 或使用 None
            return None
    userdata = MockUserData()


In [ ]:
# --- 0. API 金鑰與初始化 (省略，未更改) ---
API_KEY = userdata.get('OpenWeather')
if not API_KEY:
    print("❌ 警告：OpenWeather API Key 未在 Colab secrets 中找到。即時天氣獲取功能將受影響。")

In [ ]:
# --- 1. 設定與載入模型 (省略，未更改) ---
MODEL_PATH = '/content/random_forest_final_model.pth'
SCALER_PATH = '/content/scaler.pkl'
FEATURES_PATH = '/content/feature_names.pkl'
OPTIMAL_THRESHOLD = 0.515
DEFAULT_CITY = 'Canberra'
model = None
scaler = None
model_features = []
load_success = False

try:
    if os.path.exists(MODEL_PATH) and os.path.exists(SCALER_PATH) and os.path.exists(FEATURES_PATH):
        model = joblib.load(MODEL_PATH)
        scaler = joblib.load(SCALER_PATH)
        model_features = joblib.load(FEATURES_PATH)
        load_success = True
        print("✅ 模型與工具載入成功！")
    else:
        print("❌ 警告：部分或全部模型檔案不存在。AI 預測功能將受限。")
except Exception as e:
    print(f"❌ 載入時發生錯誤: {e}")
    model, scaler, model_features = None, None, []


In [ ]:
# --- 2. 定義選項清單 ---
LOCATIONS = ['Adelaide', 'Albany', 'Albury', 'AliceSprings', 'BadgerysCreek', 'Ballarat', 'Bendigo',
    'Brisbane', 'Cairns', 'Canberra', 'Cobar', 'CoffsHarbour', 'Dartmoor', 'Darwin', 'GoldCoast',
    'Hobart', 'Katherine', 'Launceston', 'Melbourne', 'MelbourneAirport', 'Mildura', 'Moree',
    'MountGambier', 'MountGinini', 'Newcastle', 'Nhil', 'NorahHead', 'NorfolkIsland',
    'Nuriootpa', 'PearceRAAF', 'Penrith', 'Perth', 'PerthAirport', 'Portland', 'Richmond',
    'Sale', 'SalmonGums', 'Sydney', 'SydneyAirport', 'Townsville', 'Tuggeranong', 'Uluru',
    'WaggaWagga', 'Walpole', 'Watsonia', 'Williamtown', 'Witchcliffe', 'Wollongong', 'Woomera'
]
# 新增名稱修正地圖
NAME_FIX_MAP = {
    'AliceSprings': 'Alice Springs',
    'CoffsHarbour': 'Coffs Harbour',
    'GoldCoast': 'Gold Coast',
    'MountGambier': 'Mount Gambier',
    'WaggaWagga': 'Wagga Wagga',
    # 如果這些城市在 OWM 中也無法找到，我們就使用代理
}
# 鄰近地點代理地圖 (用於 OWM API 查詢)
LOCATION_PROXIES = {
    'BadgerysCreek': 'Sydney',
    'Ballarat': 'Melbourne',
    'Bendigo': 'Melbourne',
    'Dartmoor': 'Mount Gambier',
    'Katherine': 'Darwin',
    'MelbourneAirport': 'Melbourne',
    'Mildura': 'Adelaide',
    'MountGinini': 'Canberra',
    'Nhil': 'Adelaide',
    'NorahHead': 'Newcastle',
    'NorfolkIsland': 'Sydney',
    'Nuriootpa': 'Adelaide',
    'PearceRAAF': 'Perth',
    'Penrith': 'Sydney',
    'PerthAirport': 'Perth',
    'Portland': 'Mount Gambier',
    'Richmond': 'Sydney',
    'Sale': 'Melbourne',
    'SalmonGums': 'Perth',
    'SydneyAirport': 'Sydney',
    'Tuggeranong': 'Canberra',
    'Uluru': 'Alice Springs',
    'Walpole': 'Perth',
    'Watsonia': 'Melbourne',
    'Williamtown': 'Newcastle',
    'Witchcliffe': 'Perth',
    'Woomera': 'Adelaide',
    'Albany': 'Perth'
}
WIND_DIRS = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', '其他']
CURRENT_MONTH = str(datetime.datetime.now().month)
MONTHS = [str(i) for i in range(1, 13)]

WEATHER_CONDITIONS = {
    '晴天': ('01d', '晴天'),
    '多雲': ('04d', '多雲'),
    '陰天/局部多雲': ('03d', '陰天/局部多雲'),
    '小雨': ('09d', '小雨'),
    '中雨/大雨': ('10d', '中雨/大雨'),
    '雷陣雨': ('11d', '雷陣雨'),
    '雪': ('13d', '雪'),
    '霧/霾': ('50d', '霧/霾'),
}
WEATHER_CHOICES = list(WEATHER_CONDITIONS.keys())


In [ ]:
# --- 3. OpenWeather API 函數 (省略，未更改) ---
def degree_to_compass(deg):
    if deg is None: return '其他'
    deg = deg % 360
    if 337.5 <= deg or deg < 22.5: return 'N'
    if 22.5 <= deg < 67.5: return 'NE'
    if 67.5 <= deg < 112.5: return 'E'
    if 112.5 <= deg < 157.5: return 'SE'
    if 157.5 <= deg < 202.5: return 'S'
    if 202.5 <= deg < 247.5: return 'SW'
    if 247.5 <= deg < 292.5: return 'W'
    if 292.5 <= deg < 337.5: return 'NW'
    return '其他'


In [ ]:
def format_weather_icon(icon_code, description, status_message):
    status_msg_short = status_message.split('：')[-1].split('。')[0].replace("。", "")

    if icon_code and icon_code != 'N/A' and '無法獲取' not in description:
        icon_url = f"https://openweathermap.org/img/wn/{icon_code}@2x.png"
        return f"""
        <div style="text-align: center; padding: 10px; border: 1px solid #ddd; border-radius: 8px; background-color: #f9f9f9; min-width: 180px;">
            <p style="margin: 0; font-size: 0.9em; color: #777;">今天天氣概況</p>
            <img src="{icon_url}" alt="{description}" style="width: 80px; height: 80px;">
            <h4 style="margin: 0; font-size: 1.2em; font-weight: 600; color: #333;">{description}</h4>
            <p style="margin: 5px 0 0; color: #555; font-size: 0.8em;">({status_msg_short})</p>
        </div>
        """
    else:
        return f"""
        <div style="padding: 10px; border: 1px solid #ddd; border-radius: 8px; text-align: center; color: #cc0000; background-color: #f9f9f9; min-width: 180px;">
            ❌ 無法獲取概況
            <p style="margin: 5px 0 0; font-size: 0.8em; color: #777;">({status_msg_short})</p>
        </div>
        """


In [ ]:
def calculate_sunshine_hours(sunrise_ts, sunset_ts):
    if sunrise_ts is None or sunset_ts is None:
        return 8.0
    time_diff_seconds = sunset_ts - sunrise_ts
    sunshine_hours = round(time_diff_seconds / 3600, 1)
    return sunshine_hours


In [ ]:
def get_open_weather(city_name):
    """
    從 OpenWeatherMap API 獲取天氣數據，先嘗試名稱修正，再嘗試代理查詢。
    """
    DEFAULT_ICON = 'N/A'
    DEFAULT_DESC = '無法獲取'
    DEFAULT_DATA = [15.0, 25.0, 60, 15.0, 30.0, "N", 50, 20.0, 1015.0, 0.0, 8.0, DEFAULT_ICON, DEFAULT_DESC, "API 錯誤，返回預設值。"]

    # 步驟 1: 處理 API Key 缺失或選擇 '其他'
    if not API_KEY or city_name == '其他':
        status_message = "⚠️ 預設值：由於 API Key 缺失或選擇 '其他'，返回預設天氣數據。"
        DEFAULT_DATA[-1] = status_message
        return DEFAULT_DATA

    # 步驟 2: 名稱修正 (處理 BoM vs OWM 的城市名稱差異)
    original_city = city_name
    api_query_city = NAME_FIX_MAP.get(city_name, city_name)
    proxy_used = False

    # 步驟 3: 代理檢查 (如果修正名稱後仍然是小站點，則使用代理城市)
    # 注意：這裡使用 LOCATION_PROXIES 檢查原始名稱，但查詢時使用修正後/原始名稱
    if city_name in LOCATION_PROXIES:
        api_query_city = LOCATION_PROXIES[city_name] # 覆蓋 NAME_FIX_MAP 的結果
        proxy_used = True

    url = f"https://api.openweathermap.org/data/2.5/weather?q={api_query_city},AU&appid={API_KEY}&units=metric&lang=zh_tw"

    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()

        # ... (數據解析部分保持不變) ...
        min_temp = round(data.get("main", {}).get("temp_min", DEFAULT_DATA[0]), 1)
        max_temp = round(data.get("main", {}).get("temp_max", DEFAULT_DATA[1]), 1)
        humidity_9am = data.get("main", {}).get("humidity", DEFAULT_DATA[2])
        current_temp = round(data.get("main", {}).get("temp", DEFAULT_DATA[7]), 1)
        pressure_9am = round(data.get("main", {}).get("pressure", DEFAULT_DATA[8]), 1)

        wind_speed_ms = data.get("wind", {}).get("speed", DEFAULT_DATA[3] / 3.6)
        wind_speed_kmh = round(wind_speed_ms * 3.6, 1)
        wind_gust_ms = data.get("wind", {}).get("gust", DEFAULT_DATA[4] / 3.6)
        wind_gust_kmh = round(wind_gust_ms * 3.6, 1)
        wind_deg = data.get("wind", {}).get("deg", 0)
        wind_dir = degree_to_compass(wind_deg)

        cloud_percent = data.get("clouds", {}).get("all", DEFAULT_DATA[6])
        rainfall_3h = data.get("rain", {}).get("3h", DEFAULT_DATA[9])

        sunrise_ts = data.get("sys", {}).get("sunrise")
        sunset_ts = data.get("sys", {}).get("sunset")
        sunshine_hours = calculate_sunshine_hours(sunrise_ts, sunset_ts)

        weather_info = data.get("weather", [{}])[0]
        icon_code = weather_info.get("icon", DEFAULT_ICON)
        description = weather_info.get("description", DEFAULT_DESC)

        # 狀態訊息：備註代理或名稱修正
        if proxy_used:
            # 這是代理查詢的最高優先級備註
            status_message = f"✅ 成功獲取 {original_city} 的數據 (使用代理城市：{api_query_city})。"
        elif original_city != api_query_city:
            # 這是名稱修正的備註
            status_message = f"✅ 成功獲取 {original_city} 的數據 (修正名稱查詢：{api_query_city})。"
        else:
            status_message = f"✅ 成功獲取 {original_city} 的即時數據。"

        return min_temp, max_temp, humidity_9am, wind_speed_kmh, wind_gust_kmh, wind_dir, cloud_percent, current_temp, pressure_9am, rainfall_3h, sunshine_hours, icon_code, description, status_message

    except requests.exceptions.HTTPError as errh:
        # 錯誤訊息中顯示實際查詢的城市名稱，便於用戶除錯
        status_message = f"❌ HTTP 錯誤 (查詢城市: {api_query_city})。請檢查 API Key 或城市名稱，返回預設值。"
    except Exception as e:
        status_message = f"❌ API 請求失敗: {e}。返回預設值。"

    DEFAULT_DATA[-1] = status_message
    return DEFAULT_DATA[:-1] + [status_message]

In [ ]:
# --- 4. 核心預測函數 (保持不變) ---
def predict_rain(
    location, month,
    # 9am/Manual inputs
    min_temp, max_temp, humidity_9am, wind_speed_9am, wind_gust_speed, wind_dir_9am, cloud_9am, temp_9am, pressure_9am, rainfall, sunshine,
    # 3pm inputs (Explicitly from UI)
    wind_dir_3pm, wind_speed_3pm, humidity_3pm, pressure_3pm, cloud_3pm, temp_3pm,
    # Climate Manual
    evaporation, wind_gust_dir,
    # UI Status
    icon_code, description, status_message
):
    if model is None or scaler is None or not model_features:
        formatted_icon_status = format_weather_icon(icon_code, description, status_message)
        return "❌ 錯誤：模型或所需工具未載入", "0%", "請檢查檔案路徑", formatted_icon_status

    data_map = {
        'MinTemp': min_temp, 'MaxTemp': max_temp, 'Rainfall': rainfall,
        'Evaporation': evaporation, 'Sunshine': sunshine, 'WindGustSpeed': wind_gust_speed,
        'WindDir9am': wind_dir_9am, 'WindSpeed9am': wind_speed_9am, 'Humidity9am': humidity_9am,
        'Pressure9am': pressure_9am, 'Cloud9am': cloud_9am, 'Temp9am': temp_9am,
        'WindDir3pm': wind_dir_3pm, 'WindSpeed3pm': wind_speed_3pm, 'Humidity3pm': humidity_3pm,
        'Pressure3pm': pressure_3pm, 'Cloud3pm': cloud_3pm, 'Temp3pm': temp_3pm,
        'RainToday': 1.0 if rainfall > 1.0 else 0.0,
        'WindGustDir': wind_gust_dir,
    }

    input_df = pd.DataFrame(0.0, index=[0], columns=model_features)

    try:
        for feature_name, value in data_map.items():
            if feature_name in input_df.columns and feature_name not in ['WindDir9am', 'WindDir3pm', 'WindGustDir']:
                input_df[feature_name] = float(value)

    except Exception as e:
        return f"資料輸入錯誤: {str(e)}", "0%", "請檢查輸入值的型別是否正確。", format_weather_icon(icon_code, description, status_message)

    try:
        location_feature = f'Location_{location}'
        if location_feature in input_df.columns: input_df[location_feature] = 1.0
        wind_dir_feature = f'WindDir9am_{wind_dir_9am}'
        if wind_dir_feature in input_df.columns: input_df[wind_dir_feature] = 1.0
        wind_dir_3pm_feature = f'WindDir3pm_{wind_dir_3pm}'
        if wind_dir_3pm_feature in input_df.columns: input_df[wind_dir_3pm_feature] = 1.0
        wind_gust_dir_feature = f'WindGustDir_{wind_gust_dir}'
        if wind_gust_dir_feature in input_df.columns: input_df[wind_gust_dir_feature] = 1.0
        month_feature = f'Month_{month}'
        if month_feature in input_df.columns: input_df[month_feature] = 1.0

        input_scaled = scaler.transform(input_df)
    except Exception as e:
        return f"資料標準化錯誤: {str(e)}", "0%", "請確認 Scaler 檔案正確且輸入欄位與訓練時一致。", format_weather_icon(icon_code, description, status_message)

    try:
        proba = model.predict_proba(input_scaled)[:, 1][0]
        prediction = 1 if proba >= OPTIMAL_THRESHOLD else 0

        prob_percent = f"{proba * 100:.1f}%"

        if prediction == 1:
            result_text = "🌧️ 預測結果：會下雨 (Rain)"
            detail = "機率超過閾值，建議攜帶雨具。" if proba >= OPTIMAL_THRESHOLD else "機率高，請做好準備。"
            result_icon = 'https://openweathermap.org/img/wn/10d@2x.png'
        else:
            result_text = "☀️ 預測結果：不會下雨 (No Rain)"
            detail = "預測不會下雨，安心出門！" if proba < 0.2 else "預測不會下雨，但仍建議留意天氣變化。"
            result_icon = 'https://openweathermap.org/img/wn/01d@2x.png'

        result_icon_html = f"""
        <div style="text-align: center; padding: 10px; border: 1px solid #ddd; border-radius: 8px; background-color: #e6f7ff; min-width: 150px;">
            <p style="margin: 0; font-size: 0.9em; color: #777;">模型預測概況</p>
            <img src="{result_icon}" alt="{result_text}" style="width: 80px; height: 80px;">
            <h4 style="margin: 0; font-size: 1.2em; font-weight: 600; color: #333;">{result_text.split('：')[-1]}</h4>
        </div>
        """

        formatted_icon_status = format_weather_icon(icon_code, description, status_message)

        return result_icon_html, prob_percent, detail, formatted_icon_status

    except Exception as e:
        return f"預測執行錯誤: {str(e)}", "0%", "模型在預測時發生了預期外的錯誤。", format_weather_icon(icon_code, description, status_message)


In [ ]:
# --- 5. 建立 Gradio UI ---
# --- 5.1 進階輸入欄位定義 ---
temp_9am_input = gr.Slider(minimum=-5.0, maximum=45.0, value=20.0, step=0.1, label="9點溫度 Temp9am (°C)")
humidity_9am_input = gr.Slider(minimum=0, maximum=100, value=60, step=1, label="9點濕度 Humidity9am (%)")
pressure_9am_input = gr.Number(value=1015.0, label="9點氣壓 Pressure9am (hPa)", minimum=990, maximum=1050)
wind_speed_9am_input = gr.Slider(minimum=0, maximum=100, value=15, step=1, label="9點風速 WindSpeed9am (km/h)")
wind_dir_9am_input = gr.Dropdown(choices=WIND_DIRS, value='N', label="9點風向 WindDir9am")
cloud_9am_input = gr.Slider(minimum=0, maximum=100, value=50, step=5, label="9點雲量 Cloud9am (%)")

temp_3pm_input = gr.Slider(minimum=0.0, maximum=50.0, value=25.0, step=0.1, label="3點溫度 Temp3pm (°C)")
humidity_3pm_input = gr.Slider(minimum=0, maximum=100, value=50, step=1, label="3點濕度 Humidity3pm (%)")
pressure_3pm_input = gr.Number(value=1015.0, label="3點氣壓 Pressure3pm (hPa)", minimum=990, maximum=1050)
wind_speed_3pm_input = gr.Slider(minimum=0, maximum=100, value=15, step=1, label="3點風速 WindSpeed3pm (km/h)")
wind_dir_3pm_input = gr.Dropdown(choices=WIND_DIRS, value='N', label="3點風向 WindDir3pm (預設近似 9am)")
cloud_3pm_input = gr.Slider(minimum=0, maximum=100, value=50, step=5, label="3點雲量 Cloud3pm (%)")

evaporation_input = gr.Slider(minimum=0.0, maximum=15.0, value=5.0, step=0.1, label="蒸發量 Evaporation (mm)")
sunshine_input = gr.Slider(minimum=0.0, maximum=18.0, value=8.0, step=0.1, label="日照時數 Sunshine (hr)")
wind_gust_input = gr.Slider(minimum=0, maximum=150, value=30, step=1, label="最大陣風速度 WindGustSpeed (km/h)")
wind_gust_dir_input = gr.Dropdown(choices=WIND_DIRS, value='N', label="最大陣風方向 WindGustDir")

min_temp_input = gr.Slider(minimum=-10.0, maximum=40.0, value=15.0, step=0.1, label="最低溫 MinTemp (°C)")
max_temp_input = gr.Slider(minimum=0.0, maximum=50.0, value=25.0, step=0.1, label="最高溫 MaxTemp (°C)")
rainfall_input = gr.Slider(minimum=0.0, maximum=120.0, value=0.0, step=0.1, label="降雨量 Rainfall (mm)")
INITIAL_WEATHER_HTML = format_weather_icon('N/A', '無法獲取', '等待城市選擇...')


In [ ]:
# --- Gradio UI 邏輯 ---
with gr.Blocks(theme=gr.themes.Soft(), title="澳洲降雨預測系統") as demo:
    gr.Markdown(
        """
        # 🌦️ 澳洲氣象降雨與 AI 預測系統
        ---
        """
    )

    # 隱藏欄位 (省略)
    api_icon_code = gr.State("N/A")
    api_description = gr.State("等待城市選擇...")
    api_status_message = gr.State("等待城市選擇...")

    # --- 頂部常駐區塊 ---
    with gr.Group():
        with gr.Row():
            with gr.Column(scale=2):
                gr.Markdown("### 1. 基本資料 (5 個核心輸入)")
                with gr.Row():
                    location_input = gr.Dropdown(choices=LOCATIONS, value=DEFAULT_CITY, label="地點 Location")
                    month_input = gr.Dropdown(choices=MONTHS, value=CURRENT_MONTH, label="月份 Month")

                with gr.Row():
                    min_temp_input.render()
                    max_temp_input.render()

                with gr.Row():
                    rainfall_input.render()

            with gr.Column(scale=1, min_width=200):
                gr.Markdown("### 2. 今天天氣概況 (可修改)")
                weather_icon_display = gr.HTML(label="當前天氣概況", value=INITIAL_WEATHER_HTML)
                weather_desc_manual = gr.Dropdown(choices=WEATHER_CHOICES, value=WEATHER_CHOICES[1], label="手動調整天氣概況 (會覆蓋 API 值)", interactive=True)


    # --- 進階調整區塊 (濕度輸入集中於此) ---
    with gr.Accordion("點擊展開：進階調整與所有特徵 (總共 16 個進階欄位)", open=False):

        gr.Markdown("#### 進階輸入 A: 早上 9 點觀測 (依序：溫、濕、壓、風速、風向、雲)")
        with gr.Row(): # 溫度、濕度、氣壓
            temp_9am_input.render()
            # 9am 濕度作為關鍵輸入
            humidity_9am_input.render()
            pressure_9am_input.render()
        with gr.Row(): # 風速、風向、雲量
            wind_speed_9am_input.render()
            wind_dir_9am_input.render()
            cloud_9am_input.render()

        gr.Markdown("#### 進階輸入 B: 氣候學變數 (依序：日照、蒸發量、陣風速、陣風向)")
        with gr.Row():
            sunshine_input.render()
            evaporation_input.render()
        with gr.Row():
            wind_gust_input.render()
            wind_gust_dir_input.render()

        gr.Markdown("#### 進階輸入 C: 下午 3 點預測所需數據 (依序：溫、濕、壓、風速、風向、雲)")
        with gr.Row(): # 溫度、濕度、氣壓
            temp_3pm_input.render()
            # 3pm 濕度作為關鍵輸入
            humidity_3pm_input.render()
            pressure_3pm_input.render()
        with gr.Row(): # 風速、風向、雲量
            wind_speed_3pm_input.render()
            wind_dir_3pm_input.render()
            cloud_3pm_input.render()


    # --- 函數定義與連動 ---

    def update_weather_icon(desc_key, current_status):
        icon_code, description = WEATHER_CONDITIONS.get(desc_key, ('N/A', '無法獲取'))
        new_status = f"✅ 手動設置：當前天氣概況為 {description}。"
        new_html = format_weather_icon(icon_code, description, new_status)
        return new_html, icon_code, description, new_status

    def update_weather_data(city):
        api_data = get_open_weather(city)

        # 1. 解包 9am/手動特徵
        min_temp, max_temp, humidity_9am, wind_speed_9am, wind_gust_speed, wind_dir_9am, cloud_9am, temp_9am, pressure_9am, rainfall, sunshine_hours = api_data[0:11]
        icon_code, description, status_message = api_data[11], api_data[12], api_data[13]

        # 2. 計算 3pm/衍生預設值
        humidity_3pm = int(round(humidity_9am * 0.9)) # 保持 3pm 的預設值計算

        # 移除 humidity_avg 相關邏輯

        wind_dir_3pm = wind_dir_9am
        wind_speed_3pm = wind_speed_9am
        pressure_3pm = pressure_9am
        cloud_3pm = cloud_9am
        temp_3pm = max_temp
        wind_gust_dir = wind_dir_9am

        default_weather_desc = WEATHER_CHOICES[1]
        for key, (_, desc) in WEATHER_CONDITIONS.items():
            if desc in description:
                default_weather_desc = key
                break

        formatted_icon_status = format_weather_icon(icon_code, description, status_message)

        # 調整返回的輸出列表
        return (
            min_temp, max_temp, rainfall,
            temp_9am, humidity_9am, pressure_9am, wind_speed_9am, wind_dir_9am, cloud_9am,
            sunshine_hours, evaporation_input.value, wind_gust_speed, wind_gust_dir,
            temp_3pm, humidity_3pm, pressure_3pm, wind_speed_3pm, wind_dir_3pm, cloud_3pm,
            formatted_icon_status, icon_code, description, status_message, default_weather_desc
        )

    # *** 網頁載入時自動執行一次 ***
    # 調整 initial_outputs 列表
    initial_outputs = [
        min_temp_input, max_temp_input, rainfall_input,

        temp_9am_input, humidity_9am_input, pressure_9am_input, wind_speed_9am_input, wind_dir_9am_input, cloud_9am_input,

        sunshine_input, evaporation_input, wind_gust_input, wind_gust_dir_input,

        temp_3pm_input, humidity_3pm_input, pressure_3pm_input, wind_speed_3pm_input, wind_dir_3pm_input, cloud_3pm_input,

        weather_icon_display, api_icon_code, api_description, api_status_message, weather_desc_manual
    ]

    demo.load(
        fn=update_weather_data,
        inputs=[location_input],
        outputs=initial_outputs
    )

    location_input.change(
        fn=update_weather_data,
        inputs=[location_input],
        outputs=initial_outputs
    )

    # ---------------------------------------------------------------------------------
    # 移除所有濕度連動綁定
    # ---------------------------------------------------------------------------------

    weather_desc_manual.change(
        fn=update_weather_icon,
        inputs=[weather_desc_manual, api_status_message],
        outputs=[weather_icon_display, api_icon_code, api_description, api_status_message]
    )


    # 開始預測按鈕 (省略)
    with gr.Row():
        submit_btn = gr.Button("🔮 開始預測", variant="primary", size="lg")

    # --- 預測結果區塊 (省略) ---
    gr.Markdown("### 4. 預測結果")
    with gr.Row():
        output_icon_display = gr.HTML(label="模型預測概況")
        with gr.Column(scale=2):
            output_prob = gr.Textbox(label="下雨機率 (%)", interactive=False)
            output_detail = gr.Textbox(label="建議", interactive=False)
            output_label = gr.Label(label="最終判定", visible=False)


    # 綁定預測事件
    submit_btn.click(
        fn=predict_rain,
        inputs=[
            location_input, month_input,
            # 9am inputs
            min_temp_input, max_temp_input, humidity_9am_input, wind_speed_9am_input, wind_gust_input, wind_dir_9am_input, cloud_9am_input, temp_9am_input, pressure_9am_input, rainfall_input, sunshine_input,
            # 3pm inputs
            wind_dir_3pm_input, wind_speed_3pm_input, humidity_3pm_input, pressure_3pm_input, cloud_3pm_input, temp_3pm_input,
            # Climate inputs
            evaporation_input, wind_gust_dir_input,
            api_icon_code, api_description, api_status_message
        ],
        outputs=[output_icon_display, output_prob, output_detail, weather_icon_display]
    )


In [ ]:
# --- 6. 啟動 ---
if __name__ == '__main__':
    demo.launch(share=True, debug=False)